In [1]:
import pandas as pd
import numpy as np
import itertools
import scipy.stats
import statsmodels.stats.weightstats

## 4.1 標準誤差

### 表4.2 母数の集計

In [2]:
N1 = 4; n1 = 2
x1 = np.array([165, 166, 171, 180], dtype=np.float64)
mu = np.mean(x1)
hensa = x1 - mu
hensa2 = np.power(hensa,2)
sigma2 = np.sum(hensa2)/N1
sigma = np.sqrt(sigma2)

In [3]:
mu

170.5

In [4]:
sigma

5.937171043518958

### 表4.4 標本サイズ2の全ての標本の抽出

In [5]:
xs = np.array(list(itertools.combinations(x1, 2)))
xbars = np.apply_along_axis(np.mean, 1, xs)
mub = np.mean(xbars)
hensab = xbars - mub
hensa2b = np.power(hensab, 2)
sigma2b = np.sum(hensa2b)/6
sigmab = np.sqrt(sigma2b)

In [6]:
xs

array([[165., 166.],
       [165., 171.],
       [165., 180.],
       [166., 171.],
       [166., 180.],
       [171., 180.]])

In [7]:
mub

170.5

In [8]:
xbars

array([165.5, 168. , 172.5, 168.5, 173. , 175.5])

In [9]:
sigmab

3.427827300200522

### 表4.5 標準誤差の計算

In [10]:
se0 = sigma / np.sqrt(n1)
correct = np.sqrt((N1 - n1)/(N1 - 1))
se1 = se0 * correct

In [11]:
se1

3.427827300200522

## 4.2 信頼区間

### t分布のq分位点の確認 (4.14)

In [12]:
scipy.stats.t.ppf(1-0.025, 49)

2.009575234489209

#### ファイル読み込み

In [13]:
data04 = pd.read_csv('./causality/data04.csv')
data04

,x1,y3,t1,y0,y1,y0t,y1t
0,74,76,1,NaN,76.0,68,76
1,82,75,0,75.0,NaN,75,84
2,72,75,1,65.0,NaN,65,75
3,96,84,0,84.0,NaN,84,97
4,83,75,0,NaN,84.0,75,84
5,72,74,1,65.0,NaN,65,74
6,85,76,0,NaN,87.0,76,87
7,87,77,0,77.0,NaN,77,89
8,86,77,0,77.0,NaN,77,87
9,77,80,1,NaN,80.0,70,80


#### 要約統計量

In [14]:
data04.describe()

,x1,y3,t1,y0,y1,y0t,y1t
count,20.000000,20.000000,20.000000,8.000000,12.000000,20.000000,20.000000
mean,81.950000,76.600000,0.300000,73.125000,84.083333,73.800000,83.850000
std,8.999854,5.245549,0.470162,6.812541,9.129954,7.898034,8.430989
min,58.000000,61.000000,0.000000,65.000000,61.000000,52.000000,61.000000
25%,76.250000,75.000000,0.000000,66.500000,81.500000,69.500000,79.250000
50%,83.500000,76.500000,0.000000,75.000000,85.500000,75.000000,84.500000
75%,87.250000,80.000000,1.000000,77.000000,89.250000,78.500000,89.000000
max,96.000000,87.000000,1.000000,84.000000,96.000000,87.000000,97.000000


### 4.2.3 信頼区間による対応のある場合の2標本t検定

In [15]:
n1 = len(data04)
data04['diff'] = data04['y1t'] - data04['y0t']
m1 = data04['diff'].mean()
s1 = data04['diff'].std(ddof=1)
talpha = scipy.stats.t.ppf(1-0.025, n1 - 1)

In [16]:
m1 + talpha*s1/np.sqrt(n1)

10.666325491505752

In [17]:
m1 - talpha*s1/np.sqrt(n1)

9.43367450849425

In [18]:
dsw = statsmodels.stats.weightstats.DescrStatsW(data04['diff'])
_t_test_result = dsw.ttest_mean(alternative='two-sided')
_ttest_confint = dsw.tconfint_mean(alternative='two-sided')
print(_t_test_result)
print(_ttest_confint)

pd.DataFrame(
    [*_t_test_result, *_ttest_confint],
    index=['t-statistic', 'p-value', 'df', 'confint_low', 'confint_high'],
    columns=['t-test result'],
)

(34.12951766024227, 1.6291445414422036e-18, 19.0)
(9.43367450849425, 10.666325491505752)


,t-test result
t-statistic,3.412952e+01
p-value,1.629145e-18
df,1.900000e+01
confint_low,9.433675e+00
confint_high,1.066633e+01


### 4.2.4 信頼区間による対応のない場合の2標本t検定

In [19]:
y0obs = data04['y0'].dropna()
y1obs = data04['y1'].dropna()
n0 = len(y0obs)
n1 = len(y1obs)
s0 = y0obs.std(ddof=1)
s1 = y1obs.std(ddof=1)
num = (s1**2/n1 + s0**2/n0)**2
denom = ((s1**2/n1)**2)/(n1-1) + ((s0**2/n0)**2)/(n0-1)
df1 = num/denom
xbar = y1obs.mean() - y0obs.mean()
se1 = np.sqrt(s1**2/n1 + s0**2/n0)
talpha = scipy.stats.t.ppf(1-0.025, df1)

In [20]:
xbar + talpha * se1

18.469369847637893

In [21]:
xbar - talpha * se1

3.4472968190287663

In [22]:
cm = statsmodels.stats.weightstats.CompareMeans.from_data(y1obs, y0obs)
_t_test_result = cm.ttest_ind(alternative='two-sided', usevar='unequal')
_ttest_confint = cm.tconfint_diff(alternative='two-sided', usevar='unequal')
print(_t_test_result)
print(_ttest_confint)

pd.DataFrame(
    [*_t_test_result, *_ttest_confint],
    index=['t-statistic', 'p-value', 'df', 'confint_low', 'confint_high'],
    columns=['welch-test result'],
)

(3.0692267168379623, 0.006714432861111201, 17.674073174398252)
(3.447296819028768, 18.46936984763789)


,welch-test result
t-statistic,3.069227
p-value,0.006714
df,17.674073
confint_low,3.447297
confint_high,18.469370
